In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score , classification_report , multilabel_confusion_matrix
import cv2
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,GlobalAveragePooling2D
from tensorflow.keras.activations import relu,softmax

import os
import warnings
warnings.filterwarnings('ignore')
import sys

from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
import json

In [2]:
class VGG16Model :
  def __init__(self, train_path , validation_path , model_path) :
    try :
      self.train_path = train_path
      self.validation_path = validation_path
      self.target_labels = os.listdir(self.train_path)
      self.model_path = model_path
    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')

  def ImageProcessing(self) :
    try :
      train_data_process = ImageDataGenerator(rescale=1/255 ,
                                  shear_range = 0.2 ,
                                  zoom_range = 0.2 ,
                                  horizontal_flip = True)
      val_data_process = ImageDataGenerator(rescale=1/255)

      self.train_data = train_data_process.flow_from_directory(self.train_path ,
                                                      target_size = (256,256),
                                                      classes = self.target_labels ,
                                                      class_mode = "categorical",
                                                      batch_size = 16)

      self.validation_data = val_data_process.flow_from_directory(self.validation_path ,
                                                                  classes = self.target_labels,
                                                                  target_size=(256,256),
                                                                  class_mode = "categorical")
    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')

  def Model(self) :
    try:
      v16 = VGG16(input_shape=[256,256]+[3] , weights="imagenet" , include_top=False)
      for layer in v16.layers :
        layer.trainable = False
      x = GlobalAveragePooling2D()(v16.output)
      predict = Dense(len(self.target_labels),activation='softmax')(x)
      self.model = Model(inputs = v16.inputs,outputs=predict)
      self.model.compile(optimizer="adam" , loss = "categorical_crossentropy" , metrics = ["Accuracy"])

    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')

  def Training(self) :
    try :
      self.Model()
      self.ImageProcessing()
      self.model.fit(self.train_data,
                    validation_data = self.validation_data,
                    epochs=5)

      self.model.save(f'{self.model_path}/VGG16_model.h5')

    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')


In [3]:
if __name__ == "__main__" :
  train_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Train_data"
  validation_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Validation_data"
  model_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Model"

  v16_Model = VGG16Model(train_path , validation_path , model_path)
  v16_Model.Training()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Found 5094 images belonging to 34 classes.
Found 1020 images belonging to 34 classes.
Epoch 1/5
319/319 ━━━━━━━━━━━━━━━━━━━━ 3568s 11s/step - Accuracy: 0.0875 - loss: 3.4234 - val_Accuracy: 0.2686 - val_loss: 2.9515
Epoch 2/5
319/319 ━━━━━━━━━━━━━━━━━━━━ 215s 499ms/step - Accuracy: 0.3084 - loss: 2.8899 - val_Accuracy: 0.3951 - val_loss: 2.5844
Epoch 3/5
319/319 ━━━━━━━━━━━━━━━━━━━━ 160s 501ms/step - Accuracy: 0.4120 - loss: 2.5633 - val_Accuracy: 0.4627 - val_loss: 2.3375
Epoch 4/5
319/319 ━━━━━━━━━━━━━━━━━━━━ 153s 480ms/step - Accuracy: 0.4449 - loss: 2.3593 - val_Accuracy: 0.4627 - val_loss: 2.1699
Epoch 5/5
319/319 ━━━━━━━━━━━━━━━━━━━━ 152s 475ms/step - Accuracy: 0.4598 - loss: 2.2134 - val_Accuracy: 0.5020 - val_loss: 2.0379


In [4]:
class Evaluation :
  def __init__(self , test_path , model) :
    try :
      self.model = model
      self.test_path = test_path
      self.target_labels = sorted(os.listdir(self.test_path))
      test_data_process = ImageDataGenerator(rescale=1/255)
      self.test_data = test_data_process.flow_from_directory(self.test_path,
                                                          classes = self.target_labels,
                                                          target_size=(256,256),
                                                          class_mode = "categorical")
    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')
  def Validation(self , report_path) :
    try:
      y_true = np.array(self.test_data.labels)
      y_true = to_categorical(y_true, num_classes=len(self.target_labels))
      y_pred_prob = self.model.predict(self.test_data)  # Get probability scores
      y_pred = (y_pred_prob > 0.5).astype(int)
      conf_matrices = multilabel_confusion_matrix(y_true, y_pred)

      # Summing over all labels to get overall TP, TN, FP, FN
      TP = conf_matrices[:, 1, 1].sum()
      TN = conf_matrices[:, 0, 0].sum()
      FP = conf_matrices[:, 0, 1].sum()
      FN = conf_matrices[:, 1, 0].sum()

      accuracy = (TP + TN) / (TP + TN + FP + FN)
      precision = TP/(TP+FP) if (TP + FP) > 0 else 0
      recall = TP / (TP+FN) if (TP + FN) > 0 else 0
      f1_sc = 2 * ((precision*recall) / (precision + recall)) if (precision + recall) > 0 else 0

      report = {"True Positives" : TP,
                "True Negatives" : TN,
                "Flase Positives" : FP,
                "False Negatives" : FN ,
                "Accuracy" : accuracy ,
                "Precision" : precision ,
                "Recall" : recall,
                "F1_Score" : f1_sc
                }

      def convert_numpy(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert arrays to lists
        elif isinstance(obj, (np.int64, np.int32)):
            return int(obj)  # Convert NumPy integers to Python int
        elif isinstance(obj, (np.float64, np.float32)):
            return float(obj)  # Convert NumPy floats to Python float
        else:
            return obj

      with open(report_path, "w") as json_file:
        json.dump(report, json_file, indent=4 , default=convert_numpy)


      print(f"Classification report saved successfully")

    except Exception as e :
      er_type, er_msg, line_no = sys.exc_info()
      print(f'{er_type} <-----> {er_msg} <-----> {line_no.tb_lineno}')


In [5]:
model = tensorflow.keras.models.load_model("/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Model/VGG16_model.h5")
test_path = "/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Test_data"
obj = Evaluation(test_path , model)
obj.Validation("/content/drive/MyDrive/INTERNSHIP FOLDER/task 1/Model/VGG16_validation_report.json")

Found 680 images belonging to 34 classes.
22/22 ━━━━━━━━━━━━━━━━━━━━ 344s 16s/step
Classification report saved successfully
